# partlist-io2md

A tool to create a Markdown part list from a BOM (Bill Of Materials) extracted
from a project designed with [Stud.IO CAD](https://www.bricklink.com/v3/studio/download.page).
For each submodel in the project a dedicated list is created inside the markdown file.

> MIT License
> 
> Copyright &copy; 2022 by Alessandro Varesi
> 
> Permission is hereby granted, free of charge, to any person obtaining a copy
> of this software and associated documentation files (the "Software"), to deal
> in the Software without restriction, including without limitation the rights
> to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
> copies of the Software, and to permit persons to whom the Software is
> furnished to do so, subject to the following conditions:
>
> The above copyright notice and this permission notice shall be included in all
> copies or substantial portions of the Software.
>
> THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
> IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
> FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
> AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
> LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
> OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
> SOFTWARE.
>

In [ ]:
%%capture

import sys
# Install pip packages in current Jupyter kernel
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install urllib
!{sys.executable} -m pip install beautifulsoup4

## Choose the IO file name

In next code cell set the CSV filename (with path if needed)

In [ ]:
iofile = "../../sets/10281/10281-bags.io"

Now `Run all` cells of this notebook

In [ ]:
import zipfile
import pandas as pd
from IPython.display import display, Markdown

with zipfile.ZipFile(iofile) as z:
    # print (z.infolist())

    with z.open('.info', pwd = b'soho0909') as f:
        pass
        # print (f.readlines())

    model_file = z.open('model.ldr',  pwd = b'soho0909')
    
    # for l in model_file.readlines():
    #    print (l.strip().decode('utf-8-sig'))

    model_file.seek(0)

In [ ]:
# Get color information from StudioColorDefinition.txt 
# ... this file is from Stud.io application
import math

colors = pd.read_csv("StudioColorDefinition.txt", sep='\t')
# colors

def getColorInfo(color: int):
    """Get 'BL Color Code' and 'BL Color Name'

    ### Parameter
    color : int
        the LDraw color code

    ### Returns
    bl_color : dictionary
        BL Color Code : int
            the Bricklink color code (integer)
        BL Color Name : str 
            the Bricklinr color name (str)
    
    """

    # (colors['BL Color Code'] == colors['BL Color Code']) is atrick to check is not a NaN
    r = colors.loc[(colors['LDraw Color Code'] == color) & (colors['BL Color Code'] == colors['BL Color Code'])]

    return {
        'BL Color Code': int(r.iloc[0]['BL Color Code']),
        'BL Color Name': r.iloc[0]['BL Color Name']
    }

# colors = colors[['LDraw Color Code','BL Color Code','BL Color Name']]

getColorInfo(84)

In [ ]:
# Scan model file and fill the BOM dictionary structure
# part = { 
# }
submodels = {}
parts = {}

class Model:
    def __init__(self, name: str):
        self.name = name
        self.BOM = {}
        self.submodels = {}

    def addSubmodel(self, sm_name: str):
        if sm_name not in submodels:
            submodels[sm_name] = Model(sm_name)
        self.submodels[sm_name] = submodels[sm_name]

    def addPart(self, part: str, color: int):
        part_id = f'{color}_{part}'
        if part_id not in parts:
            parts[part_id] = LegoPart(part, color)
        if part_id in self.BOM:
            self.BOM[part_id]['qty'] += 1
        else:
            self.BOM[part_id]={'name': part, 'color': color, 'qty': 1}

class LegoPart:
    def __init__(self, partname: str, color: int):
        self.partname = partname
        self.color = color
        c = getColorInfo(color)
        self.blcolor = c['BL Color Code'] 
        self.colorname = c['BL Color Name']


l = model_file.readline().strip().decode('utf-8-sig').split(' ')

model = Model(' '.join(l[2:]))
# throw away next 4 lines
model_file.readline()
model_file.readline()
model_file.readline()

print (f'Creating BOM for model : {model.name}')

thisModel = model

for l in model_file.readlines():

    line = l.strip().decode('utf-8-sig').split(' ')
    # check for line type 
    if line[0] == '0':
        if line[1] == 'FILE':
            sm_name = " ".join(line[2:])
            if sm_name not in submodels:
                submodels[sm_name] = Model(sm_name)
            thisModel = submodels[sm_name]
            print (f'Create submodel : {thisModel.name}')
    elif line[0] == '1':
        color = int(line[1]) # color
        part = " ".join(line[14:]) # part
        if color == 16:    # this is a submodel in the BOM
            thisModel.addSubmodel(part)
        elif line[1] == 24:
            pass
        else:
            thisModel.addPart(part, color)

for p in parts.values():
    print(f'{p.partname} - Color: {p.color} ({p.colorname})')
    
    

In [ ]:
import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup

partname = '3036'

req = urllib.request.Request(f'https://www.bricklink.com/v2/catalog/catalogitem.page?P={partname}', headers={'User-Agent': 'Mozilla/5.0'})
# getpage = requests.get(f'https://www.bricklink.com/v2/catalog/catalogitem.page?P=3036')
html = urllib.request.urlopen(req, timeout=10).read()
soup = BeautifulSoup(html, 'html.parser')

description = soup.find('span', {'id': 'item-name-title'})

print(description)

In [ ]:
md = "Picture | Qty | Code | Description | Color |\n"
md += "--------|----:|------|-------------|-------|\n"
for i in df.itertuples():
    item = i.BLItemNo
    qty = i.Qty
    description = i.PartName
    color = i.ColorName
    colorid = i.BLColorId
    image = f"![{item}](https://img.bricklink.com/P/{colorid:.0f}/{item}.jpg)"
    link = f"[{item}](https://www.bricklink.com/v2/catalog/catalogitem.page?P={item}&idColor={colorid:.0f})"

    md += f"{image}| {qty:.0f} | {link} | {description} | {color}\n"

display(Markdown(md))

In [ ]:
outmd = "## Here is your markdown block\n"
outmd += "Copy next code block and paste in your _md_ file\n"
outmd += "```(copy)\n"
outmd += md
outmd += "```\n"

display(Markdown(outmd))